## ForJoe

*Some things from my notebook that would be good to have recorded somewhere.*

Outline:
 - overall goal and scope of the project
 - overview of verification
 - previous attempts
 - next attempts

The goal of the project is to verify properties of a closed loop neural network controller. The problem has multiple facets and subproblems to deal with, but the overall objective is to prove a formal property on a system that has this structure:

                input state -> [control NN + dynamics NN] -> output state -> repeat

Or in other words: 
$$ x_t \longrightarrow C(x_t) = c \longrightarrow D(x_t, c) \longrightarrow x_{t+1}$$

That said, this is just one way to think about it in our particular case. The more general problem is simply to show that $ f_t(X_0) \subseteq Y $ for some starting set $X_0$, where $f_t$ denotes $t$ applications of $f$. If this property is true, the controller can be said to be stable (subject to the set $Y$, of course). 

We also want to demonstrate this with an image-based controller that uses an image of the state. I.e.:

$$ x_t \longrightarrow I(x_t) \longrightarrow C(I(x_t)) = c \longrightarrow D(x_t, c) \longrightarrow x_{t+1}$$

The added challenge there is that the mapping $I$ is not one-to-one with $x$. I.e. nearby states can map to the same image.

The particular environment we're working on is the inverted pendulum, since it's simple and is a classic reinforcement learning problem (found in OpenAIGym, etc.). The exact architecture of the network is up in the air. In particular, the control network can be fully-connected or convolutional as well as feed-forward or recurrent (ideally, we want to prove a property on all permutations thereof, but not necessarily the same property). 

The exact architecture of the problem is a bit more nuanced than described above due to how the dynamics are overapproximated. See [Chelsea's paper](https://stanford.app.box.com/file/478687098549) for details. In particular, the output of the network is not $x_{t+1}$ in this scheme, but an overapproximated set $X_{t+1}$. That said, for properties that use only a single timestep, this nuance can be temporarily neglected I think.

## Tools
Just to note, while we can use **NeuralVerification.jl** if we work in julia, that package is probably not best suited for the more complex networks we want to show this on, and does not implement the state-of-the-art algorithms. So it's probably not for the best.

So far, Chelsea has mostly used **Marabou** — a verification tool written in C++ (with a python interface) that is the next generation of Reluplex, and the current state of the art for complete solvers. A complete solver is one which does not (over)approximate and therefore never returns false negatives (never states that a property is violated when in fact it is not). While this is useful, it often slows computation down, so we'll see if we can relax it in some cases and still prove properties. Marabou is based on SAT/SMT solvers, which under the hood ultimately maps to solving linear programs, often with some other logic on top. You can check out a bunch of Chelsea's stuff for how to use Marabou with python, including this straightforward test file: https://github.com/guykatzz/Marabou/blob/master/maraboupy/test/multi_input_test.py (if you can't reach it it's because the marabou repo is private... we'll have to get you access).

There is another state-of-the-art solver called **ERAN** from ETH-Zurich which is reachability-based. It is not complete like Marabou but could afford other advantages for some formulations of the problem (it also seems to have a python interface). https://github.com/eth-sri/eran


## Property in state space

In the pendulum problem, the state consists of angle and angular velocity, $x = (\theta, \dot\theta)$ and the control network may output a torque $\ddot\theta$. A finite time safety property in this space could be something like: 
$$X_0 = \{x \mid \theta \in [a, b],\  \dot\theta \in [c, d])\}$$
$$f_t(X_0) \subseteq Y$$ 
for a known safe set $Y$. Notably, if $Y$ is a subset of $X_0$, the safety property holds for all time. I.e., if the property reads,

$$f_t(X_0) = X_1 \subseteq X_0,$$

then subsequent applications of f on $X_1$ are also guaranteed to so be subsets of $X_0$, since $X_1$ itself is a subset of $X_0$.

Likewise, if at any $n>0,\ f_{t+n}(X_0) \subseteq f_{t}(X_0)$ then we have a cycle, and therefore stability. This is identical to the previous infinite-time property with $f_t(X^*_0) = X_0$. This fact has been guiding the majority of my thinking on the topic, and I'll say more about it later.

## Properties in general

Note that in the example above, $\theta$ and $\dot\theta$ take on interval values. I mentioned in our conversation that input sets need to be convex for verification. Technically this is not true in general (it is in NV.jl though). I think in the most general case, Marabou can handle any conjunctions or disjunctions of linear constraints on the input and output sets. I'm not terribly familiar with it still, but from Chelsea's code, it looks like setting upper and lower bounds on variables is the primary interface to creating the verification problem. From earlier discussions, I seem to recall that certain logic may also be used (e.g. if-then), but that could have been manual tweeking Chelsea was working with on top of the Marabou interface; not sure, have to defer to Chelsea on that front. In any case, linear constraints should be safe to use in pretty much every context, and intervals are linear constraints, so we're good.

The verification problem for a SAT based solver is to find a point 
$$x \in X \mid f(x) \in Y^c$$
I.e. a point in $X$ that maps to the complement of the desired set. As I understand Marabou (not that well), the set you encode as the output set is $Y^c$, and it simply tries to solve a feasibility problem on the sets $X$ and $Y^c$ using SMT methods. If the problem has a feasible solution, the safety property is violated.

### $Y^c$ as a $\cap$ of linear constraints (convex)

These are polytopes. When $Y^c$ is a polytope, the property can be verified by solving a single satisfiability problem. If 
$$\exists x \in X \mid f(x) \in Y^c,$$
you're done (and the safety property is violated). This kind of propery maps to an "exclusion zone". I.e. a convex region of the state space you *don't* want to be in. These types of problems can be encoded into a single satisfiability problem.

### $Y^c$ as a $\cup$ of linear constraints (complement of convex)

Complement of a polytope. This maps to an "inclusion zone" safety property. I.e. a convex region of the state space you *do* want to be in. In this case, $Y^c$ is not convex, and $Y$ is. Defining $Y^c$ as 
$$ Y^c = \bigcup y^c_i, $$

Where each constraint $y^c_i$ is of the form $a^T_i x \leq b_i$

the property is violated if there exists an assigment for $x$ such that:

$$a^T_1 f(x) > b_1 \ \ \lor\ \  a^T_2 f(x) > b_2 \dots \ \lor\ \  a^T_n f(x) > b_n$$

Which actually means you need to solve $n$ of these satisfiability problems, one for each constraint.

### both $\cap$ and $\cup$ of linear constraints 

Can be broken up into smaller problems of the forms above.

## Forward invariance example 

# disregard all of this for now. It was written a while ago and all the notation and concepts are jumbled up by now

Consider the input set that is off by up to 10 degrees in either direction of vertical, i.e. $X_0 = [-10^{\circ}, 10^{\circ}]$. We can hypothesize that this set is forward invariant under the network. I.e. that $f(X_0) \subseteq X_0$. However, we must consider $\dot\theta$ in this assessment, and clearly this property does not hold over all $\dot\theta$. Additionally, even if we show that it holds for $\dot\theta_0 = 0$, this does not help unless we find that $\dot\theta_1 = 0$ in all cases (unlikely). Depending on the verification scheme taken for this problem, you can go one of several ways.
- ground truth $\dot\theta$

If you add a variable that denotes ground-truth velocity (probably a good place to start) then the input velocity set can simply be a range. However, positive and negative angular velocites behave differently on opposite sides of the input set $X_0$. I.e. positive velocities on the positive side push the pendulum away from equilibrium, but the same velocity on the opposite side pushes towards equilibrium. As a result, it's unlikely to find a property that applies to all of $X_0$. Instead, you could split up $X_0$ into $X_0^+$ and $X_0^-$. On $X_0^+$, you could test the property $f(X_0^+,~\dot\theta \in [-a, 0]) \subseteq \{X_0,~A^+\}$ which actually has a good chance of holding. Then the equivalent property on $X_0^-$ is $f(X_0^-,~\dot\theta \in [0, a]) \subseteq \{X_0,~A^-\}$. If both properties hold, then you can guarantee a property on all of $X_0$ that applies for all time; from both properties holding, we can infer that in the worst case, the sets hop back and forth indefinitely.

- inferring velocity from images

If you go with inferring the velocity by passing in multiple images (4 is common) then velocity space is a kind of by-product of the images passed in. I'm honestly not sure how you could represent a set of velocities as well as states without the two sets of information interfering. 


All of the above does not mention convolutions or image-space or anything like that. It would be perfectly alright to start by verifying those properties on a network which takes in $[\theta, \dot\theta]$ directly, since this is a challenging enough problem. The image issue adds to layers of complexity. The first one is obvious, and that is the network size. instead of O(10) nodes we have O(10k), which even state of the art solvers have a hard time with. The other issue, which is perhaps more pernicious, is dealing with input/output sets. A set which is naturally convex in state space is not in image space. For example, how can you represent the set of images corresponding to the angles $X_0 = [-\pi/36, \pi/36]$? You can't do it in a compat or convex way. When verifying properties on images, most people resort to $L_1$ or $L_{\infty}$ norm properties, by turning pixels on and off or adding noise to them, but these don't have physical meaning like the kind we're looking for. We can do something similar however, by also applying the same technique as before. We can consider the set $I(X_0^+)$ to be the set for which any pixels within some (probably small) arc are above a threshold, and all others are off. While this set does not represent a range of angles exactly, it does consist of all posible pixel activations in the range of allowed angles, and is a good place to start (most importantly, it is naturally convex). It may even be possible to restrict the exact number of active and inactive pixels in this set, or other such modifications, while keeping the problem representation the same